# Working with Time Zones

## Python's Time Zone Model

The `tzinfo` class is an abstract base class, and you are required to implement three methods:

```python
class tzinfo:
    def tzname(self, dt):
        raise NotImplementedError()

    def utcoffset(self, dt):
        raise NotImplementedError()
        
    def dst(self, dt):
        raise NotImplementedError()
        
    ...
```

- `tzname`: Return the name or abbreviation for the time zone at the given datetime.
- `utcoffset`: Return a `datetime.timedelta` representing the offset from UTC at the given datetime.
- `dst`: Return a `datetime.timedelta` representing the amount of the daylight saving time offset at a given time. *This method is rarely useful*.

The reason these are *methods* rather than fixed values is that a `tzinfo` represents a mapping between a datetime and the set of rules that applies at that time in this zone. The same concrete `tzinfo` class is used for many `datetime`s.

### Example: Eastern Time

In [1]:
from dateutil import tz
from datetime import datetime, timedelta

In [2]:
def print_tzinfo(dt):
    """Convenience function for printing the time zone information of a datetime"""
    print("dt.tzname: " + str(dt.tzname()))
    print("dt.utcoffset: " + str(dt.utcoffset() / timedelta(hours=1)) + " hours")
    print("dt.dst: " + str(dt.dst() / timedelta(hours=1)) + " hours")

In [3]:
EASTERN = tz.gettz("America/New_York")

In [4]:
dt_std = datetime(2019, 3, 1, 12, tzinfo=EASTERN)

In [5]:
print_tzinfo(dt_std)

dt.tzname: EST
dt.utcoffset: -5.0 hours
dt.dst: 0.0 hours


In [6]:
dt_dst = datetime(2019, 4, 1, 12, tzinfo=EASTERN)

In [7]:
print_tzinfo(dt_dst)

dt.tzname: EDT
dt.utcoffset: -4.0 hours
dt.dst: 1.0 hours


### Exercise: Implement a UTC time zone class

The simplest `tzinfo` classes to implement are fixed offsets from UTC and UTC itself, which have offsets that do not change as a function of `datetime`. To practice a basic tzinfo implementation, implement a class `UTC` representing the UTC time zone.

In [8]:
from datetime import tzinfo

In [9]:
from datetime import timezone

In [10]:
datetime(2019, 1, 1, tzinfo=timezone.utc).dst()

In [11]:
class UTC(tzinfo):
    def tzname(self, dt):
        return "UTC"
    
    def utcoffset(self, dt):
        return timedelta(0)
    
    def dst(self, dt):
        return None

In [12]:
# Tests
test_cases = [
    datetime(1, 1, 1),
    datetime(1857, 2, 7),
    datetime(1970, 1, 1),
    datetime(2010, 3, 21, 2, 16)
]

utc = UTC()
for dt in test_cases:
    dt_act = dt.replace(tzinfo=utc)
    dt_exp = dt.replace(tzinfo=timezone.utc)
    
    assert dt_act.tzname() == dt_exp.tzname(), f'tznames must match for {dt}'
    assert dt_act.utcoffset() == dt_exp.utcoffset(), f'utcoffset must match for {dt}'
    assert dt_act.dst() == dt_exp.dst(), f'dst must match for {dt}'

## Creating and working with time-zone objects

The standard way to create a datetime literal is to attach it to the constructor by passing it to the `tzinfo` argument of the constructor:

In [13]:
from helper_functions import print_dt_tzinfo

In [14]:
dt = datetime(2017, 8, 11, 14, tzinfo=tz.gettz('America/New_York'))
print_dt_tzinfo(dt)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


If you have a naïve wall time or a wall time in another zone that you want to translate without shifting, use `datetime.replace`:

In [15]:
print_dt_tzinfo(dt.replace(tzinfo=tz.gettz('America/Los_Angeles')))

2017-08-11 14:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


If you have an *absolute* time, in UTC or otherwise, and you want to represent it in another timezone, use `datetime.astimezone`:

In [16]:
print_dt_tzinfo(dt.astimezone(tz.gettz('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


### `pytz`

In `pytz`, `datetime.astimezone()` still works as expected:

In [17]:
import pytz

In [18]:
print_dt_tzinfo(dt.astimezone(pytz.timezone('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


But the constructor and `.replace` methods fail horribly:

In [19]:
print_dt_tzinfo(dt.replace(tzinfo=pytz.timezone('America/Los_Angeles')))

2017-08-11 14:00:00-0753
    tzname:   LMT;      UTC Offset:  -7.88h;        DST:      0.0h


This is because `pytz` uses a different time zone model. `pytz`'s time zone model implements the `tzinfo` interface *statically*, which is to say that the `tzname`, `utcoffset` and `dst` are all calculated eagerly, when the `tzinfo` is attached to the `datetime`.

In order to accomplish this, `pytz` expects all `tzinfo` objects to be attached to the `datetime` *by the time zone object itself*, using the `localize()` method:

In [20]:
EASTERN_p = pytz.timezone('America/New_York')
dt_p = EASTERN_p.localize(datetime(2017, 8, 11, 14))
print_dt_tzinfo(dt_p)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


This also means that unlike with normal `tzinfo` objects, after you've done some arithmetic on a `pytz`-aware `datetime` object, you must `normalize` it:

In [21]:
print('dateutil:')
print_dt_tzinfo(dt + timedelta(days=180))
print('')
print('pytz')
print_dt_tzinfo(dt_p + timedelta(days=180))

dateutil:
2018-02-07 14:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

pytz
2018-02-07 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


In [22]:
print_dt_tzinfo(EASTERN_p.normalize(dt_p + timedelta(days=180)))

2018-02-07 13:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


### `datetime.now` and `datetime.fromtimestamp`

The alternate constructors `now()` and `fromtimestamp` both take an optional argument `tz`:


```python
    def now(tz=None):
        """Get the datetime representing the current time"""
        # ...

    def fromtimestamp(self, timestamp, tz=None):
        """ Return the datetime corresponding to a POSIX timestamp """
        # ...
```

If the `tz` parameter is not passed, they will return a *naïve* datetime, representing the time in your system local time.

In [23]:
datetime.fromtimestamp(1577854800.0)

datetime.datetime(2020, 1, 1, 0, 0)

If you want an *aware* timezone from a timestamp (or the current time), pass it to the `tz` parameter, and it will calculate the correct `datetime` matching that time zone. This works with both `pytz` and `dateutil.tz`:

In [24]:
print("pytz:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN_p))
print("")
print("dateutil:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN))

pytz:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

dateutil:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


**Note**: There are also the semi-deprecated `datetime.utcnow()` and `datetime.utcfromtimestamp()` functions. These  return a *naïve* datetime expressed in UTC. It is almost always better to simply pass `tz=UTC` (where `UTC` is some time zone object), which will give you an *aware* `datetime`.

### Exercise: Current time in multiple time zones


To practice using time zones, try writing a function that takes as inputs a list of IANA time zones and prints the time in each time zone.

So, for example:

```python
>>> now_in_zones(['Asia/Tokyo',
...               'Europe/Berlin',
...               'America/New_York',
...               'America/Los_Angeles'])
```
```
Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00
```

In [25]:
def now_in_zones(tz_list):
    # For now I'll fib and pretend the current datetime is 2020
    dt_utc = datetime(2020, 1, 1, tzinfo=tz.gettz('America/New_York'))
    for tzstr in tz_list:
        dt = dt_utc.astimezone(tz.gettz(tzstr))
        print(f"{tzstr + ':':<25} {dt}")
        
now_in_zones(['Asia/Tokyo',
              'Europe/Berlin',
              'America/New_York',
              'America/Los_Angeles'])
    

Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00


## Ambiguous and imaginary times

### Ambiguous times
An *ambiguous* time is when the same "wall time" occurs more than once, such as during a DST to STD transition.

In [26]:
NYC = tz.gettz('America/New_York')

In [27]:
dt1 = datetime(2004, 10, 31, 4, 30, tzinfo=tz.UTC)
for i in range(4):
    dt = (dt1 + timedelta(hours=i)).astimezone(NYC)
    ambig_str = 'Ambiguous' if tz.datetime_ambiguous(dt) else 'Unambiguous'
    print(f'{dt} | {dt.tzname()} |  {ambig_str}')

2004-10-31 00:30:00-04:00 | EDT |  Unambiguous
2004-10-31 01:30:00-04:00 | EDT |  Ambiguous
2004-10-31 01:30:00-05:00 | EST |  Ambiguous
2004-10-31 02:30:00-05:00 | EST |  Unambiguous


### [PEP 495: Local Time Disambiguation](https://www.python.org/dev/peps/pep-0495/)
- First introduced in Python 3.6
- Introduces the `fold` attribute of `datetime`
- Changes aware datetime comparison around ambiguous times

PEP 495 makes whether you are on the fold side a *property of the `datetime`*:

In [28]:
print_dt_tzinfo(datetime(2004, 10, 31, 1, 30, tzinfo=NYC))
print('')
print_dt_tzinfo(datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=NYC))

2004-10-31 01:30:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h

2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


**Note**: `fold=1` represents the *second* instance of an ambiguous `datetime`.

### Imaginary times
*Imaginary* times are wall times that never occur in a given time zone, such as during an STD to DST transition:

In [29]:
dt1 = datetime(2004, 4, 4, 6, 30, tzinfo=tz.UTC)
for i in range(3):
    dt = (dt1 + timedelta(hours=i)).astimezone(NYC)
    print(f'{dt} | {dt.tzname()} ')

2004-04-04 01:30:00-05:00 | EST 
2004-04-04 03:30:00-04:00 | EDT 
2004-04-04 04:30:00-04:00 | EDT 


### Handling ambiguous times

Both `dateutil` and `pytz` will automatically give you the right *absolute time* if converting from an absolute time.

In [30]:
dt1 = datetime(2004, 10, 31, 6, 30, tzinfo=tz.UTC)   # This is in the fold in EST

In [31]:
# dateutil
dt_du = dt1.astimezone(tz.gettz('America/New_York'))
print(repr(dt_du))
print_dt_tzinfo(dt_du)

datetime.datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=tzfile('/usr/share/zoneinfo/America/New_York'))
2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


In [32]:
# pytz
dt_pytz = dt1.astimezone(pytz.timezone('America/New_York'))
print(repr(dt_pytz))    # Note that pytz doesn't set the fold attribute
print_dt_tzinfo(dt_pytz)

datetime.datetime(2004, 10, 31, 1, 30, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)
2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


For backwards compatibility, `dateutil` provides a `tz.enfold` method to add a `fold` attribute if necessary:

In [33]:
dt = datetime(2004, 10, 31, 1, 30, tzinfo=NYC)
tz.enfold(dt)

datetime.datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=tzfile('/usr/share/zoneinfo/America/New_York'))

```python
Python 2.7.12
Type "help", "copyright", "credits" or "license" for more information.
>>> from datetime import datetime
>>> from dateutil import tz
>>> dt = datetime(2004, 10, 31, 1, 30, tzinfo=tz.gettz('US/Eastern'))
>>> tz.enfold(dt)
_DatetimeWithFold(2004, 10, 31, 1, 30, tzinfo=tzfile('/usr/share/zoneinfo/US/Eastern'))
>>> tz.enfold(dt).tzname()
'EST'
>>> dt.tzname()
'EDT'
```

To detect ambiguous times, `dateutil` provides `tz.datetime_ambiguous`:

In [34]:
tz.datetime_ambiguous(datetime(2004, 10, 31, 1, 30, tzinfo=NYC))

True

In [35]:
tz.datetime_ambiguous(datetime(2004, 10, 31, 1, 30), NYC)

True

In [36]:
dt_0 = datetime(2004, 10, 31, 0, 30, tzinfo=NYC)
for i in range(3):
    dt_i = dt_0 + timedelta(hours=i)
    dt_i = tz.enfold(dt_i, tz.datetime_ambiguous(dt_i))
    print(f'{dt_i} (fold={dt_i.fold})')

2004-10-31 00:30:00-04:00 (fold=0)
2004-10-31 01:30:00-05:00 (fold=1)
2004-10-31 02:30:00-05:00 (fold=0)


**Note:** `fold` is ignored when `datetime` is not ambiguous:

In [37]:
for i in range(3):
    dt_i = tz.enfold(dt_0 + timedelta(hours=i), fold=1)
    print(f'{dt_i} (fold={dt_i.fold})')

2004-10-31 00:30:00-04:00 (fold=1)
2004-10-31 01:30:00-05:00 (fold=1)
2004-10-31 02:30:00-05:00 (fold=1)


### Handling imaginary times

While functions that convert from an *absolute time* to another absolute time (e.g. `astimezone`, `fromtimestamp` and `now`) will never create an imaginary time, it is possible to create imaginary times with functions that manipulate the naïve portion of the date, such as the constructor, arithmetic and `replace`.

In order to determine if you have created an imaginary time after one of these operations, you can use `dateutil`'s `tz.datetime_exists()`:

In [38]:
dt_0 = datetime(2004, 4, 4, 1, 30, tzinfo=NYC)
for i in range(3):
    dt = dt_0 + timedelta(hours=i)
    print(f'{dt} ({{}})'.format('Exists' if tz.datetime_exists(dt) else 'Imaginary'))

2004-04-04 01:30:00-05:00 (Exists)
2004-04-04 02:30:00-04:00 (Imaginary)
2004-04-04 03:30:00-04:00 (Exists)


For the most part, if you are creating imaginary times, you want to "skip forward", to what the time *would be* if the transition had not happened: for example, if the datetime you have created is the result of an addition operation. For this, `dateutil` provides the `tz.resolve_imaginary` function:

In [39]:
dt = datetime(2004, 4, 4, 1, 30, tzinfo=NYC)
dt_imag = dt + timedelta(hours=1)   # 2004-04-04 02:30 is imaginary
print(f"Imaginary: {dt_imag}")
print(f"Resolved:  {tz.resolve_imaginary(dt_imag)}")

Imaginary: 2004-04-04 02:30:00-04:00
Resolved:  2004-04-04 03:30:00-04:00


This works for imaginary times other than 1 hour as well:

In [40]:
dt = datetime(1994, 12, 31, 9, tzinfo=tz.gettz('Pacific/Kiritimati'))
print(f'{dt} ({{}})'.format('Exists' if tz.datetime_exists(dt) else 'Imaginary'))

1994-12-31 09:00:00-10:00 (Imaginary)


In [41]:
tz.resolve_imaginary(dt)

datetime.datetime(1995, 1, 1, 9, 0, tzinfo=tzfile('/usr/share/zoneinfo/Pacific/Kiritimati'))

All of these functions for handling ambiguous and imaginary times will work for `pytz` time zones as well, though `pytz` has its own way of handling this.

### `pytz`'s approach to handling ambiguous and imaginary times
#### Ambiguous times

`pytz` predates PEP 495, and in many ways its design was intended to solve the ambiguous and imaginary time problem before there was support for doing so in the standard library. Much of its non-standard interface is a consequence of the fact that without eagerly calculating time zone offsets, there is no way to specify which ambiguous wall time your `datetime` represents.

When localizing an ambiguous `datetime`, `pytz` will default to the *second* occurrence (i.e. the `fold=1` state, usually standard time):

In [42]:
NYC_pytz = pytz.timezone('America/New_York')
dt_pytz = NYC_pytz.localize(datetime(2004, 10, 31, 1, 30))
print_dt_tzinfo(dt_pytz)

2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


To get the *first* occurrence of a given wall time (i.e. the `fold=0` state, usually daylight saving time), pass `is_dst=True` to the `localize` function:

In [43]:
dt_pytz = NYC_pytz.localize(datetime(2004, 10, 31, 1, 30), is_dst=True)
print_dt_tzinfo(dt_pytz)

2004-10-31 01:30:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


If you want to *detect* ambiguous times, pass `is_dst=None`, and `pytz` will raise an `AmbiguousTimeError` if a datetime is ambiguous:

In [44]:
for hour in (0, 1):
    dt = datetime(2004, 10, 31, hour, 30)
    try:
        NYC_pytz.localize(dt, is_dst=None)
        print(f'{dt} | Unambiguous')
    except pytz.AmbiguousTimeError:
        print(f'{dt} | Ambiguous')

2004-10-31 00:30:00 | Unambiguous
2004-10-31 01:30:00 | Ambiguous


#### Imaginary times
When using `localize` on an imaginary `datetime`, `pytz` will create an imaginary time and assign it an offset based on `is_dst`:

In [45]:
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30), is_dst=True))
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30), is_dst=False))

2004-04-04 02:30:00-04:00
2004-04-04 02:30:00-05:00


The default for `is_dst` is `False`:

In [46]:
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30)))

2004-04-04 02:30:00-05:00


Again, setting `is_dst=None` will cause `pytz` to throw an error, this time `NonExistentTimeError`:

In [47]:
dt_0 = datetime(2004, 4, 4, 1, 30)
for i in range(3):
    try:
        dt = NYC_pytz.localize(dt_0 + timedelta(hours=i), is_dst=None)
        exist_str = 'Exists'
    except pytz.NonExistentTimeError:
        exist_str = 'Imaginary'

    print(f'{dt} ({exist_str})')

2004-04-04 01:30:00-05:00 (Exists)
2004-04-04 01:30:00-05:00 (Imaginary)
2004-04-04 03:30:00-04:00 (Exists)


### Exercise: Build a `pytz`-style exception-based localizer with `dateutil`

While `dateutil`'s interface for handling imaginary and ambiguous times is compatible with `pytz` zones, there is no built-in mechanism to throw *exceptions* when imaginary or ambiguous times are created. As an exercise, try to build a `localize` function that throws an exception using `dateutil.tz`'s functions for handling ambiguous and imaginary datetimes.

In [48]:
class AmbiguousTimeError(Exception):
    """Raised if an ambiguous time is detected"""

class NonExistentTimeError(Exception):
    """Raised if an imaginary time is detected"""
    
def localize(dt, tzi, is_dst=False):
    """
    Mimicks `pytz`'s `localize` function using the `fold` attribute.
    """
    if dt.tzinfo is not None:
        raise ValueError('localize can only be used with naive datetimes')

    if is_dst is None:
        # If is_dst is None, we want to raise an error for uncertain situations
        dt_out = dt.replace(tzinfo=tzi)
        if tz.datetime_ambiguous(dt_out):
            raise AmbiguousTimeError(f"Ambiguous time {dt} in zone {tzi}")
        elif not tz.datetime_exists(dt_out):
            raise NonExistentTimeError(f"Time {dt} does not exist in zone {tzi}")
    else:
        dt_out = dt.replace(fold=(not is_dst), tzinfo=tzi)

    return dt_out

In [49]:
# Tests
from contextlib import contextmanager

@contextmanager
def assert_raises(err_type):
    try:
        yield
    except err_type:
        pass
    else:
        raise AssertionError(f"Failed to raise {err_type} exception")
        
def assert_dt_equal(dt1, dt2):
    """
    datetime equality is a bit more complicated than it may seem when dealing
    with ambiguous and imaginary datetimes
    """
    
    fail_msg = f"{dt1} != {dt2}"
    assert dt1.astimezone(tz.UTC) == dt2.astimezone(tz.UTC), fail_msg
    assert dt1.tzname() == dt2.tzname(), fail_msg
    assert dt1.utcoffset() == dt2.utcoffset(), fail_msg
    assert dt1.dst() == dt2.dst(), fail_msg
        
NYC = tz.gettz('America/New_York')
with assert_raises(AmbiguousTimeError):
    localize(datetime(2004, 10, 31, 1, 30), NYC, is_dst=None)
    
with assert_raises(NonExistentTimeError):
    localize(datetime(2004, 4, 4, 2, 30), NYC, is_dst=None)

with assert_raises(ValueError):
    localize(datetime(2004, 10, 31, 1, 30, tzinfo=NYC), NYC)
    
assert_dt_equal(
    localize(datetime(2004, 10, 31, 1, 30), NYC, is_dst=False),
    datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=NYC))

assert_dt_equal(
    localize(datetime(2004, 10, 31, 1, 30), NYC, is_dst=True),
    datetime(2004, 10, 31, 1, 30, fold=0, tzinfo=NYC))

print("Passed!")

Passed!


## Aware datetime comparison semantics

We will start this section off with a mysterious bug involving the comparison of datetimes:

In [50]:
LON = tz.gettz('Europe/London')

# Construct a datetime
x = datetime(2007, 3, 25, 1, 9, tzinfo=LON)
ts = x.timestamp()    # Get a timestamp representing the same datetime

# Get the same datetime from the timestamp
y = datetime.fromtimestamp(ts, LON)

# Get thesame datetime from the timestamp with a fresh instance of LON
z = datetime.fromtimestamp(ts, tz.gettz.nocache('Europe/London'))

In [51]:
x == y

False

In [52]:
x == z

True

In [53]:
y == z

True

To summarize: x, y and z should all represent the same `datetime` – they all have the same time zone, and y and z are the result of converting x into a timestamp and then back into a `datetime`, but for some reason `x != y`, and, even more curiously, `x == z`, even though the only difference between y and z is that z uses a different `tzinfo` object (representing the same zone). Even stranger, the equality relationship between the three is non-transitive, because `x != y` even though `x == z` and `y == z`. What the hell is going on?

### Inter-zone vs. intra-zone comparisons

In Python, comparisons between datetimes are divided into "same zone" and "different zone" comparisons. When two `datetime`s are in the same time-zone, they are equal if their "wall time" is the same, *even if they represent different absolute times*:

In [54]:
dt1 = datetime(2017, 10, 29, 1, 30, tzinfo=LON)
dt2 = datetime(2017, 10, 29, 1, 30, fold=1, tzinfo=LON)

print(f"{dt1} | {dt1.timestamp()}")
print(f"{dt2} | {dt2.timestamp()}")

2017-10-29 01:30:00+01:00 | 1509237000.0
2017-10-29 01:30:00+00:00 | 1509240600.0


In [55]:
dt1 == dt2

True

Comparisons between datetimes in *different* zones, however, use "absolute time" semantics, meaning that the wall time can be different, as long as they refer to the same time in UTC:

In [56]:
dt1 = datetime(2018, 4, 7, 1, 30, tzinfo=tz.gettz("America/Chicago"))
dt2 = datetime(2018, 4, 7, 2, 30, tzinfo=tz.gettz("America/New_York"))
print(f"{dt1} | {dt1.timestamp()}")
print(f"{dt2} | {dt2.timestamp()}")

2018-04-07 01:30:00-05:00 | 1523082600.0
2018-04-07 02:30:00-04:00 | 1523082600.0


In [57]:
dt1 == dt2

True

#### What does it mean to be in the "same zone"?

When deciding what is the "same zone" and what is a "different zone", Python uses *object equality* rather than *value equality*, which means to say two datetimes are in the same zone if `dt1.tzinfo is dt2.tzinfo`, and otherwise they are in different zones, *even if* `dt1.tzinfo == dt2.tzinfo`!

This means that we can re-cast the "intra-zone" comparison as an "inter-zone" comparison by just getting a new object to represent the same zone:

In [58]:
dt1 = datetime(2017, 10, 29, 1, 30, tzinfo=LON)
dt2 = datetime(2017, 10, 29, 1, 30, fold=1, tzinfo=tz.gettz.nocache("Europe/London"))

print(f"{dt1} | {dt1.timestamp()}")
print(f"{dt2} | {dt2.timestamp()}")
print("")
print(f"{dt1} == {dt2}: {dt1 == dt2}")

2017-10-29 01:30:00+01:00 | 1509237000.0
2017-10-29 01:30:00+00:00 | 1509240600.0

2017-10-29 01:30:00+01:00 == 2017-10-29 01:30:00+00:00: False


### Solving the mystery

Returning to the mystery we started the section off with, we can now apply the same/different zone comparison semantics to our non-transitive equality and see what's happening.

In [59]:
def zone_semantics(dt1, dt2):
    return "Same Zone" if dt1.tzinfo is dt2.tzinfo else "Different Zone"

print(f"x == y ({zone_semantics(x, y)}): {x == y}")
print(f"x: {x} ({x.timestamp()})")
print(f"y: {y} ({y.timestamp()})")
print("")
print(f"x == z ({zone_semantics(x, z)}): {x == z}")
print(f"x: {x} ({x.timestamp()})")
print(f"z: {z} ({z.timestamp()})")
print("")
print(f"y == z ({zone_semantics(y, z)}): {y == z}")
print(f"y: {y} ({y.timestamp()})")
print(f"z: {z} ({z.timestamp()})")

x == y (Same Zone): False
x: 2007-03-25 01:09:00+01:00 (1174781340.0)
y: 2007-03-25 00:09:00+00:00 (1174781340.0)

x == z (Different Zone): True
x: 2007-03-25 01:09:00+01:00 (1174781340.0)
z: 2007-03-25 00:09:00+00:00 (1174781340.0)

y == z (Different Zone): True
y: 2007-03-25 00:09:00+00:00 (1174781340.0)
z: 2007-03-25 00:09:00+00:00 (1174781340.0)


#### But wait...
Aren't `y` and `z` supposed to be `datetime.fromtimestamp(x.timestamp())`? Why is the wall time different after a round trip through a timestamp?

And if these are all in the exact same time zone, why are there two different wall times for the same timestamp?

In [60]:
tz.datetime_exists(x)

False

... because `x` is an imaginary datetime!

## Aware datetime arithmetic semantics

In [61]:
import tz_answers
from tz_answers import wall_add, wall_sub
from tz_answers import absolute_add, absolute_sub
from tz_answers import AbsoluteDateTime, WallDateTime

We have looked at *comparison* semantics, but it is worth noting that the difference between "same zone" and "different zone" comparisons really only makes a difference during *ambiguous* times and as we've seen in *imaginary* times. At all other times, the two are equivalent.

Comparison is just a specific case of the more general arithmetical property of `datetime` objects that operations between datetimes in the same zone use "wall time" semantics (i.e. looking at the clock on the wall) and operations between dateimes in different zones use "absolute time" semantics (i.e. looking at a stopwatch or timestamp).


#### Wall time vs. absolute time semantics
Colloquially, time periods tend to be overloaded concepts, and what you mean when you say, e.g., "a day" or "a month" depends on the context. Looking at the following code, what would you expect the value for `dt2` to be?

In [62]:
dt1 = datetime(2018, 3, 10, 13, tzinfo=NYC)
dt2 = dt1 + timedelta(days=1)

There are two options, the first is using "wall time" semantics, returning the next day at the same time (rolling the clock forward by 24 hours):

In [63]:
print(dt1)
print(wall_add(dt1, timedelta(days=1)))

2018-03-10 13:00:00-05:00
2018-03-11 13:00:00-04:00


The second option is to use "absolute time" semantics, where we jump forward to the point in time after which 24 hours have elapsed in the "real world". Because the start date here is immediately before a daylight saving time transition, these give *different* answers:

In [64]:
print(dt1)
print(absolute_add(dt1, timedelta(days=1)))

2018-03-10 13:00:00-05:00
2018-03-11 14:00:00-04:00


The concept of "add one day" is overloaded in this situation because the meaning of "1 day" can either mean "the period between two identical clock times on subsequent days" or it can mean "the period during which 24 hours have elapsed".

**Question**: Which behavior is more intuitive? What would you choose for the default?

Now look at subtraction between two datetimes across a DST transition:

In [65]:
dt1 = datetime(2018, 3, 10, 13, 30, tzinfo=NYC)
dt2 = datetime(2018, 3, 11, 8, 30, tzinfo=NYC)

In [66]:
print(wall_sub(dt2, dt1))

19:00:00


In [67]:
print(absolute_sub(dt2, dt1))

18:00:00


A total of 18 real hours have elapsed, but `8 - 13 % 24 == 19`, so 19 "wall hours" have elapsed.

**Question**: Which behavior is more intuitive? What would you choose as the default (ignoring your choice for the earlier question)


### Python's behavior

Assuming Python wanted to be consistent and have all operations use *either* wall-time operations or absolute-time operations, whatever you choose is likely to have some non-intuitive behaviors. If, for example, we were to choose to always use *absolute* `datetime`s, you would likely find someone who writes some variation of this code:

In [68]:
DAY = timedelta(days=1)
dtstart = AbsoluteDateTime(2018, 3, 9, 12, tzinfo=NYC)
for i in range(4):
    print(dtstart + i * DAY)

2018-03-09 12:00:00-05:00
2018-03-10 12:00:00-05:00
2018-03-11 13:00:00-04:00
2018-03-12 13:00:00-04:00


and they would be confused as to why all of a sudden their noon meeting was happening at 1 PM after a DST transition. If you choose *wall* times for everything, you'll get someone who writes some variation on *this* code:

In [69]:
dtstart = WallDateTime(2018, 4, 17, 12, tzinfo=NYC)
dtend = WallDateTime(2018, 4, 17, 12, tzinfo=tz.gettz('America/Los_Angeles'))

In [70]:
print(dtstart)
print(dtend)

2018-04-17 12:00:00-04:00
2018-04-17 12:00:00-07:00


In [71]:
print(dtend - dtstart)

0:00:00


and they'll be surprised to find that these two events that happened 3 hours apart give 0 hours difference!

As we saw in the comparison case, Python actually uses hybrid semantics that can be somewhat confusing, though there is at least a rhyme and a reason behind it. For arithmetic within the "same zone", all operations use wall time semantics. Because addition returns a `datetime` in the same time zone as the input `datetime`, it is always considered a "same zone" operation, and thus always uses wall time semantics.

Because wall time semantics across different time zones is essentially *never* what you want, the choice of "absolute time" semantics for subtractions between two datetimes in different time zones also makes sense and gives you the "intuitive" behavior you would expect.

However, note that both wall-time subtraction and absolute-time subtraction return a `timedelta`, which has no notion of whether it represents a period of "wall time" or a period of "absolute time". As a result, you are stuck with a counter-intuitive result:

In [72]:
dt1 = datetime(2018, 3, 11, 1, tzinfo=NYC)
dt2 = datetime(2018, 3, 11, 1, tzinfo=tz.gettz('America/Los_Angeles'))

In [73]:
dt1 == dt2 + (dt1 - dt2)

True

In [74]:
dt2 == dt1 + (dt2 - dt1)

False

Across a DST boundary, you cannot reverse a between-zone boundary by adding the result (a `timedelta`) to the original value!

### Exercise: Implement explicit wall-time and absolute-time arithmetic


Knowing how Python handles aware time zone arithmetic, by paying attention to the time zones you work in you should be able to design your system so that you get the semantics you want, *but* it is occasionally the case that you will mostly want wall-time semantics but for some operations you want absolute time, or vice-versa. In this case, you will want a way to explicitly operate using either wall-time or absolute-time semantics.

In this exercise, implement the `wall_add`, `wall_sub`, `absolute_add` and `absolute_sub` functions that explicitly give you wall time or absolute time semantics.

In [75]:
def wall_add(dt: datetime, offset: timedelta) -> datetime:
    """Addition with "wall-time" semantics"""
    ### REPLACE BETWEEN THESE ###
    return tz_answers.wall_add(dt, offset)
    ######### END ###############

def wall_sub(dt: datetime, other: datetime) -> timedelta:
    """Subtraction with "wall time" semantics"""
    ### REPLACE BETWEEN THESE ###
    return tz_answers.wall_sub(dt, other)
    ######### END ###############

    return dt.replace(tzinfo=None)- other.replace(tzinfo=None)

def absolute_add(dt: datetime, offset: timedelta) -> datetime:
    """Addition with "absolute time" semantics"""
    ### REPLACE BETWEEN THESE ###
    return tz_answers.absolute_add(dt, offset)
    ######### END ###############

def absolute_sub(dt: datetime, other: datetime) -> timedelta:
    ### REPLACE BETWEEN THESE ###
    return tz_answers.absolute_sub(dt, other)
    ######### END ###############

In [76]:
# Tests
sub_pairs = [
    (datetime(2018, 3, 11, 1, tzinfo=tz.gettz('America/Los_Angeles')),
     datetime(2018, 3, 11, 1, tzinfo=NYC)),
    (datetime(2018, 3, 11, 8, 30, tzinfo=NYC),
     datetime(2018, 3, 10, 13, 30, tzinfo=NYC)),
]

for dt1, dt2 in sub_pairs:
    assert absolute_sub(dt1, dt2) == tz_answers.absolute_sub(dt1, dt2), f"absolute_sub({dt1}, {dt2})"
    assert wall_sub(dt1, dt2) == tz_answers.wall_sub(dt1, dt2), f"wall_sub({dt1}, {dt2})"

add_pairs = [
    (datetime(2018, 3, 10, 13, tzinfo=NYC), timedelta(days=1)),
]

for dt, off in add_pairs:
    assert absolute_add(dt, off) == tz_answers.absolute_add(dt, off), f"absolute_add({dt}, {off})"
    assert wall_add(dt, off) == tz_answers.wall_add(dt, off), f"wall_sub({dt}, {off})"

print("Passed!")

Passed!


## Handling local times

"Local time" is a very common concept that is irritatingly difficult to capture correctly. A naïve `datetime.datetime` in Python has a somewhat overloaded meaning in that it mostly represents an abstract datetime for use with calendrical calculations, but when used as a concrete time, it is interpreted as being in the system's local time zone.

For example:

In [77]:
from helper_functions import TZEnvContext  # Sets the local time zone during the context
                                           # This does not work on Windows

dt = datetime(2020, 1, 1, 12)
with TZEnvContext("America/Los_Angeles"):
    dt_la = dt.astimezone(timezone.utc)
    
with TZEnvContext("Asia/Tokyo"):
    dt_tok = dt.astimezone(timezone.utc)
    
print(dt_la)
print(dt_tok)

2020-01-01 20:00:00+00:00
2020-01-01 03:00:00+00:00


This works in the other direction as well:

In [78]:
dt = datetime(2020, 1, 1, 21, tzinfo=tz.gettz("Asia/Tokyo"))

with TZEnvContext("UTC"):
    print(dt.astimezone(None))

2020-01-01 12:00:00+00:00


These are not true local times, however, as they do not expose any information about the time zone through `tzname()` or `utcoffset()`:

In [79]:
print(datetime(2020, 1, 1, 12).utcoffset())

None


And you cannot perform comparisons or arithmetic between naive datetimes and aware datetimes:

In [80]:
try:
    datetime(2020, 1, 1, 12) - datetime(2020, 1, 1, 12, tzinfo=NYC)
except TypeError as e:
    print(repr(e))

TypeError("can't subtract offset-naive and offset-aware datetimes")


### `dateutil.tz.tzlocal`

The Python standard library provides hooks into the operating system's time zone information in the `time module`:

In [81]:
import time

with TZEnvContext("America/New_York"):
    print(f"tzname: {time.tzname}")
    print(f"timezone: {time.timezone}")
    print(f"altzone: {time.altzone}")

tzname: ('EST', 'EDT')
timezone: 18000
altzone: 14400


But there is no concrete local time object in the standard library, *so* `dateutil` has implemented one with `dateutil.tz.tzlocal`!

In [82]:
with TZEnvContext("America/Los_Angeles"):
    print("Los Angeles")
    print_dt_tzinfo(datetime(2020, 1, 1, 12, tzinfo=tz.tzlocal()))
print("")
with TZEnvContext("America/Chicago"):
    print("Chicago")
    print_dt_tzinfo(datetime(2020, 1, 1, 12, tzinfo=tz.tzlocal()))


Los Angeles
2020-01-01 12:00:00-0800
    tzname:   PST;      UTC Offset:  -8.00h;        DST:      0.0h

Chicago
2020-01-01 12:00:00-0600
    tzname:   CST;      UTC Offset:  -6.00h;        DST:      0.0h


This allows you to get a proper timezone-aware datetime in your system's locale.

### Changing local zone during a program's run

The system time zone changing during a program's run is *not* particularly well-supported operation, and you should avoid it if at all possible when working with local times: prefer to get the desired time zone from your users rather than from the system.

On Linux, it is necessary to call `time.tzset()` after any change to the the system time locale in order to see those changes reflected in the system. On Windows, `time.tzset()` does nothing, and restarting the interpreter is the only way to update the time zone from the perspective of the `time` function.

If your program is or could run on Windows, it is preferable to use the `dateutil.tz.tzwinlocal()` function to represent local times, as it queries the system registry directly.

On Windows, you can see this problem: 

```python
>>> dt = datetime(2014, 2, 11, 17, 0)

>>> print(dt.replace(tzinfo=tz.tzlocal()).tzname())
Eastern Standard Time

>>> print(dt.replace(tzinfo=tz.win.tzwinlocal()).tzname())
Eastern Standard Time

>>> with TZWinContext('Pacific Standard Time'):
...     print(dt.replace(tzinfo=tz.tzlocal()).tzname())
...     print(dt.replace(tzinfo=tz.win.tzwinlocal()).tzname())
```
```
Eastern Standard Time
Pacific Standard Time
```

However, in both cases (unlike the situation with all other time zone types), it is preferable to make a *new `tzinfo` object for every `datetime`*. The reason for this is that a certain amount of the behavior of `tzlocal` and `tzwinlocal` are set at construction time, so:

In [83]:
with TZEnvContext("America/New_York"):
    LOCAL = tz.tzlocal()

with TZEnvContext("America/Los_Angeles"):
    print(datetime(2014, 2, 11, 17, tzinfo=LOCAL))

with TZEnvContext("America/Chicago"):
    print(datetime(2014, 2, 11, 17, tzinfo=LOCAL))


2014-02-11 17:00:00-05:00
2014-02-11 17:00:00-05:00


Note that both of these are using the offset specified in the *original* time zone. What's worse is that because `tzlocal` does not have direct access to the function mapping `datetime` to offsets, if the system time zone changes, the offsets are baked in at runtime, but the dates of DST changes use the *current system offsets*, thus creating a hybrid object that is *really* wrong:

In [84]:
with TZEnvContext("America/New_York"):
    print(datetime(2019, 3, 20, 10, tzinfo=LOCAL))
    
with TZEnvContext("Europe/London"):
    print(datetime(2019, 3, 20, 10, tzinfo=LOCAL))
    print(datetime(2019, 3, 20, 10, tzinfo=tz.gettz("Europe/London")))

2019-03-20 10:00:00-04:00
2019-03-20 10:00:00-05:00
2019-03-20 10:00:00+00:00


If possible, I recommend avoiding using "local" time at all, and instead either convert to UTC eagerly (if dealing in absolute times) or have a user-configurable time zone mapping to either the IANA database or the Windows time zone settings.

And I will note that these particular kinds of bugs are probably incredibly rare with some mitigations, since it requires a non-Windows user to change their time zone during the lifetime of a `tzlocal` object to another time zone that has a different schedule for DST changes. It can happen, but it will probably not make your application useless if you don't handle it correctly.